In [ ]:
DEPENDENCIES = " ".join(
    ['torch', 'transformers', 'sentencepiece', 'protobuf', 'gradio', 'python-pptx', 'qrcode', 'accelerate']
)

In [ ]:
%pip install {DEPENDENCIES}

import gradio as gr

import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

import colorsys
import matplotlib.pyplot as plt
import numpy as np

from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE
from pptx.util import Inches
from pptx.util import Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT

import qrcode
from io import BytesIO

In [ ]:
INFERENCE_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_PATH = 'openlm-research/open_llama_3b_v2'

BASE_COLORS = {
    'Ярко-голубой' : "#00BFFF",
    'Энергичный зеленый': "#32CD32",
    'Интенсивный розовый': "#FF1493"
}
RESOLUTIONS = ['16x9', '4x3']
PROP_REGIME_WIDTH = {'4x3': 10.0, '16x9': 13.3333}
PROP_REGIME_HEIGHT = {'4x3': 7.5, '16x9': 7.5}

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
model = LlamaForCausalLM.from_pretrained(
    MODEL_PATH, torch_dtype=torch.float16, device_map=INFERENCE_DEVICE,
)

In [ ]:
def rgb_to_hex(r, g, b):
    return "#{:02X}{:02X}{:02X}".format(r, g, b)

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return r, g, b

def create_itten_palette(base_color,analogous_koef = 0.083, complementary_koef = 0.333):
    # Convert hex color to RGB values
    r, g, b = int(base_color[1:3], 16) / 255, int(base_color[3:5], 16) / 255, int(base_color[5:7], 16) / 255

    
    h, s, v = colorsys.rgb_to_hsv(r, g, b)
    
    # Calculate complementary color
    h_comp = (h + 0.5) % 1
    r_comp, g_comp, b_comp = colorsys.hsv_to_rgb(h_comp, s, v)

    # Calculate analogous colors
    h_analog1 = (h + 0.083) % 1
    r_analog1, g_analog1, b_analog1 = colorsys.hsv_to_rgb(h_analog1, s, v)

    h_analog2 = (h - 0.083) % 1
    r_analog2, g_analog2, b_analog2 = colorsys.hsv_to_rgb(h_analog2, s, v)

    # Calculate split complementary colors
    h_split_comp1 = (h + 0.333) % 1
    r_split_comp1, g_split_comp1, b_split_comp1 = colorsys.hsv_to_rgb(h_split_comp1, s, v)

    h_split_comp2 = (h - 0.333) % 1
    r_split_comp2, g_split_comp2, b_split_comp2 = colorsys.hsv_to_rgb(h_split_comp2, s, v)

    color_wheel = [
        (r, g, b),
        (r_comp, g_comp, b_comp),
        (r_analog1, g_analog1, b_analog1),
        (r_analog2, g_analog2, b_analog2),
        (r_split_comp1, g_split_comp1, b_split_comp1),
        (r_split_comp2, g_split_comp2, b_split_comp2)
    ]

    return color_wheel

In [ ]:
def generate_title_slide(
    prs: Presentation, base_color: str, strong_color: str, weak_color: str, prop_regime: str
) -> Presentation:    
    # Add a new slide
    slide = prs.slides.add_slide(prs.slide_layouts[6])  # Choose a layout

    # Add a rectangle shape that covers the entire slide

    #------------------Градиент--------------------------
    left = top = 0
    width = prs.slide_width
    height = prs.slide_height

    rect = slide.shapes.add_shape(
        MSO_SHAPE.RECTANGLE,
        left, top, width, height
    )

    # Apply gradient fill to the rectangle shape
    fill = rect.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(*hex_to_rgb(base_color))  # Start color
    fill.gradient()

    gradient_stops = fill.gradient_stops
    gradient_stops[0].color.rgb = RGBColor(*hex_to_rgb(strong_color))  # Start color сильный цвет
    gradient_stops[1].color.rgb = RGBColor(*hex_to_rgb(base_color))  # End color средний цвет


    gradient_stops[0].position = 0.5   
    gradient_stops[1].position = 0.0


    fill.gradient_angle = 45  # Gradient direction in degrees


    #---------------Название проекта---------------------------


    txt_cont = 'Bright Tone Pitch'

    left_textbox = Inches(3/17*PROP_REGIME_WIDTH[prop_regime])
    top_textbox = Inches(4.5/13*PROP_REGIME_HEIGHT[prop_regime])
    width_textbox = Inches(11/17*PROP_REGIME_WIDTH[prop_regime])
    height_textbox = Inches(4/13*PROP_REGIME_HEIGHT[prop_regime])

    textbox = slide.shapes.add_textbox(left_textbox, top_textbox, width_textbox, height_textbox)
    frame = textbox.text_frame
    frame.word_wrap = True

    # Добавление параграфа в текстовый блок
    p = frame.add_paragraph()
    run = p.add_run()
    run.text = txt_cont
    run.font.size = Pt(80*11/len(txt_cont))  # Размер шрифта
    run.font.name = "Tahoma"  # Название шрифта
    p.font.bold = True
    p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
    run.font.color.rgb = RGBColor(*hex_to_rgb("#F5F5F5"))  # Цвет текста (белый)




    #----------------Слоган-------------------

    txt_cont = 'Seeking brevity, with an eye for detail'

    left_textbox = Inches(3/17*PROP_REGIME_WIDTH[prop_regime])
    top_textbox = Inches(7/13*PROP_REGIME_HEIGHT[prop_regime])
    width_textbox = Inches(11/17*PROP_REGIME_WIDTH[prop_regime])
    height_textbox = Inches(3/13*PROP_REGIME_HEIGHT[prop_regime])

    textbox = slide.shapes.add_textbox(left_textbox, top_textbox, width_textbox, height_textbox)
    frame = textbox.text_frame
    frame.word_wrap = True

    # Добавление параграфа в текстовый блок
    p = frame.add_paragraph()
    run = p.add_run()
    run.text = txt_cont
    run.font.size = Pt(75*11/len(txt_cont))  # Размер шрифта
    run.font.name = "Tahoma"  # Название шрифта
    p.font.italic = True
    p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
    run.font.color.rgb = RGBColor(*hex_to_rgb("#F5F5F5"))  # Цвет текста (белый)




    #-------------------------------------------

    txt_cont = 'ADAM© development team'

    left_textbox = Inches(3/17*PROP_REGIME_WIDTH[prop_regime])
    top_textbox = Inches(10/13*PROP_REGIME_HEIGHT[prop_regime])
    width_textbox = Inches(11/17*PROP_REGIME_WIDTH[prop_regime])
    height_textbox = Inches(2/13*PROP_REGIME_HEIGHT[prop_regime])

    textbox = slide.shapes.add_textbox(left_textbox, top_textbox, width_textbox, height_textbox)
    frame = textbox.text_frame
    frame.word_wrap = True

    # Добавление параграфа в текстовый блок
    p = frame.add_paragraph()
    run = p.add_run()
    run.text = txt_cont
    run.font.size = Pt(30*11/len(txt_cont))  # Размер шрифта
    run.font.name = "Tahoma"  # Название шрифта
    p.font.bold = True
    p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
    run.font.color.rgb = RGBColor(*hex_to_rgb("#F5F5F5"))  # Цвет текста (белый)
    
    return prs

In [ ]:
def generate_base_slide(
    prs: Presentation, 
    strong_color: str, 
    weak_color: str, 
    base_color: str, 
    prop_regime: str, 
    title: str, 
    content: str
) -> Presentation:    
    
    # Add a new slide
    slide = prs.slides.add_slide(prs.slide_layouts[6])  # Choose a layout

    #цвет фона

    slide_master = prs.slide_master

    background = slide_master.background
    fill = background.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(*hex_to_rgb("#F5F5F5"))

    #----------------Градиент----------------------------

    left = Inches(0)
    top = Inches(0)
    width = Inches(PROP_REGIME_WIDTH[prop_regime])
    height = Inches(3/13*PROP_REGIME_HEIGHT[prop_regime])

    rect = slide.shapes.add_shape(
        MSO_SHAPE.RECTANGLE,
        left, top, width, height
    )

    # Apply gradient fill to the rectangle shape
    fill = rect.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(*hex_to_rgb(weak_color))  # Start color
    fill.gradient()

    gradient_stops = fill.gradient_stops
    gradient_stops[0].color.rgb = RGBColor(*hex_to_rgb(base_color))  # Start color средний цвет
    gradient_stops[1].color.rgb = RGBColor(*hex_to_rgb(weak_color))  # End color слабый цвет



    gradient_stops[0].position = 0.5   
    gradient_stops[1].position = 0.0


    fill.gradient_angle = 45 


    #---------------Заголовок---------------------------


    txt_cont = title

    left_textbox = Inches(1.75/20*PROP_REGIME_WIDTH[prop_regime])
    top_textbox = Inches(0.65/13*PROP_REGIME_HEIGHT[prop_regime])
    width_textbox = Inches(17/20*PROP_REGIME_WIDTH[prop_regime])
    height_textbox = Inches(1.5/13*PROP_REGIME_HEIGHT[prop_regime])

    textbox = slide.shapes.add_textbox(left_textbox, top_textbox, width_textbox, height_textbox)
    frame = textbox.text_frame
    frame.word_wrap = True

    # Добавление параграфа в текстовый блок
    p = frame.add_paragraph()
    run = p.add_run()
    run.text = txt_cont
    run.font.size = Pt(48)  # Размер шрифта
    run.font.name = "Tahoma"  # Название шрифта
    p.font.bold = True
    p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
    run.font.color.rgb = RGBColor(*hex_to_rgb("#F5F5F5"))




    #----------------Текст-------------------

    txt_cont = content

    left_textbox = Inches(1.5/20*PROP_REGIME_WIDTH[prop_regime])
    top_textbox = Inches(3/13*PROP_REGIME_HEIGHT[prop_regime])
    width_textbox = Inches(17/20*PROP_REGIME_WIDTH[prop_regime])
    height_textbox = Inches(9/13*PROP_REGIME_HEIGHT[prop_regime])

    textbox = slide.shapes.add_textbox(left_textbox, top_textbox, width_textbox, height_textbox)
    frame = textbox.text_frame
    frame.word_wrap = True

    # Добавление параграфа в текстовый блок
    p = frame.add_paragraph()
    run = p.add_run()
    run.text = txt_cont
    run.font.size = Pt(30)  # Размер шрифта
    run.font.name = "Tahoma"  # Название шрифта
    p.font.italic = False
    p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
    run.font.color.rgb = RGBColor(*hex_to_rgb("#16161D"))  # Цвет текста (белый)
    
    return prs

In [ ]:
def generate_contacts_slide(
    prs: Presentation, 
    strong_color: str, 
    weak_color: str, 
    base_color: str, 
    prop_regime: str, 
    email: str,
    phone: str,
    website: str
) -> Presentation:
    
    # Add a new slide
    slide = prs.slides.add_slide(prs.slide_layouts[6])  # Choose a layout

    #цвет фона

    slide_master = prs.slide_master

    background = slide_master.background
    fill = background.fill
    fill.solid()
    fill.fore_color.rgb = RGBColor(*hex_to_rgb("#F5F5F5"))

    #---------------Заголовок---------------------------


    txt_cont = 'Contacts'

    left_textbox = Inches(1.75/20*PROP_REGIME_WIDTH[prop_regime])
    top_textbox = Inches(0.65/13*PROP_REGIME_HEIGHT[prop_regime])
    width_textbox = Inches(17/20*PROP_REGIME_WIDTH[prop_regime])
    height_textbox = Inches(1.5/13*PROP_REGIME_HEIGHT[prop_regime])

    textbox = slide.shapes.add_textbox(left_textbox, top_textbox, width_textbox, height_textbox)
    frame = textbox.text_frame
    frame.word_wrap = True

    # Добавление параграфа в текстовый блок
    p = frame.add_paragraph()
    run = p.add_run()
    run.text = txt_cont
    run.font.size = Pt(48)  # Размер шрифта
    run.font.name = "Tahoma"  # Название шрифта
    p.font.bold = True
    p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
    run.font.color.rgb = RGBColor(*hex_to_rgb(strong_color))




    #----------------Текст-------------------


    txt_cont = f'''Phone: {phone} \n
    Email: {email}'''

    left_textbox = Inches(1.5/20*PROP_REGIME_WIDTH[prop_regime])
    top_textbox = Inches(4/13*PROP_REGIME_HEIGHT[prop_regime])
    width_textbox = Inches(17/20*PROP_REGIME_WIDTH[prop_regime])
    height_textbox = Inches(9/13*PROP_REGIME_HEIGHT[prop_regime])

    textbox = slide.shapes.add_textbox(left_textbox, top_textbox, width_textbox, height_textbox)
    frame = textbox.text_frame
    frame.word_wrap = True

    # Добавление параграфа в текстовый блок
    p = frame.add_paragraph()
    run = p.add_run()
    run.text = txt_cont
    run.font.size = Pt(30)  # Размер шрифта
    run.font.name = "Tahoma"  # Название шрифта
    p.font.italic = False
    p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
    run.font.color.rgb = RGBColor(*hex_to_rgb("#16161D"))  # Цвет текста (белый)


    #---------------QR---Ссылка на сайт--------------------------


    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(website)  # Установите нужный URL или текст
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")

    # Сохранение изображения в байтовом формате
    img_byte_array = BytesIO()
    img.save(img_byte_array, format="PNG")
    img_byte_array.seek(0)

    # Вставка изображения QR-кода на слайд
    left = Inches(13/20*PROP_REGIME_WIDTH[prop_regime])
    top = Inches(4/13*PROP_REGIME_HEIGHT[prop_regime])
    width = Inches(6/20*PROP_REGIME_WIDTH[prop_regime])


    pic = slide.shapes.add_picture(img_byte_array, left, top, width)



    #--------гиперссылка-----------------

    left_textbox = Inches(13/20*PROP_REGIME_WIDTH[prop_regime])
    top_textbox = Inches((4/13 + 9/20)*PROP_REGIME_HEIGHT[prop_regime])
    width_textbox = Inches(6/20*PROP_REGIME_WIDTH[prop_regime])
    height_textbox = Inches(1/20*PROP_REGIME_WIDTH[prop_regime])

    textbox = slide.shapes.add_textbox(left_textbox, top_textbox, width_textbox, height_textbox)
    text_frame = textbox.text_frame

    # Добавление параграфа с гиперссылкой в текстовый блок
    p = text_frame.add_paragraph()
    p.text = website.lstrip("https://").lstrip("http://")
    p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER  # Центрирование текста в параграфе

    # Добавление гиперссылки к параграфу
    r = p.runs[0]
    hyperlink = r.hyperlink
    hyperlink.address = "website"  # Установите вашу гиперссылку

    # Настройка стиля текста
    font = r.font
    font.size = Pt(18)
    font.bold = True
    font.color.rgb = RGBColor(*hex_to_rgb(base_color))  # Средний цвет
    
    return prs

In [ ]:
def generate_text_block(project_description: str, output_type: str) -> dict:
    prompts_by_output_type = {
        "Problem": "answer, what is the key problem this service solves?",
        "Target audience": "answer, what is the target audience of this service?",
        "Value proposition": "answer, what is the value proposiion of this service?",
        "Description": "summarise the description of this service",
        "Solution": "describe, how the service works",
        "Competitors": "point its key competitors",
        "Founders": "point, who are its founders and investors?",
        "Investments direction": "imagine, what features can be added to service?",
        "Roadmap": "imagine the roadmap of service development"
    }
    print(prompts_by_output_type.keys())
    prompt = f"Text: {project_description}\nGiven this information, {prompts_by_output_type[output_type]}\n"
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(INFERENCE_DEVICE)

    generation_output = model.generate(
        input_ids=input_ids, max_new_tokens=32
    )
    return tokenizer.decode(generation_output[0])[len(prompt)+2:].rstrip('</s>')


In [ ]:
def generate_presentation(
    base_color: str, 
    prop_regime: str, 
    text_blocks: dict[str, str], 
    email: str, 
    phone: str, 
    website: str
):
    prs = Presentation()
    
    itten_palette = create_itten_palette(base_color)
    
    strong_color = rgb_to_hex(*list(map(lambda x: int(x*255),itten_palette[2])))
    weak_color = rgb_to_hex(*list(map(lambda x: int(x*255),itten_palette[3])))
    base_color = rgb_to_hex(*list(map(lambda x: int(x*255), itten_palette[0])))
    
    new_slide_width = Inches(PROP_REGIME_WIDTH[prop_regime])  # Новая ширина слайда (пример: 10 дюймов)
    new_slide_height = Inches(PROP_REGIME_HEIGHT[prop_regime])  # Новая высота слайда (пример: 7.5 дюймов)
    prs.slide_width = new_slide_width
    prs.slide_height = new_slide_height
    
    prs = generate_title_slide(prs, strong_color, weak_color, base_color, prop_regime)
    
    for block in text_blocks:
        prs = generate_base_slide(
            prs, strong_color, weak_color, base_color, prop_regime, block, text_blocks[block]
        )
    
    prs = generate_contacts_slide(
        prs, strong_color, weak_color, base_color, prop_regime, email, phone, website
    )
    
    prs.save("presentation.pptx")

In [ ]:
def generate_pitch_deck(
    project_description: str, 
    email: str, 
    phone: str, 
    website: str, 
    base_color: str, 
    resolution: str
) -> str:
    
    output_types = [
        'Problem', 
        'Target audience', 
        'Value proposition', 
        'Description', 
        'Solution', 
        'Competitors', 
        'Founders', 
        'Investments direction', 
        'Roadmap'
    ]
    project_description = project_description.rstrip()
    pitch_deck_text_blocks = {
        output_type: generate_text_block(project_description, output_type) 
        for output_type in output_types
    }
    
    generate_presentation(
        BASE_COLORS[base_color], 
        resolution, 
        pitch_deck_text_blocks, 
        email, 
        phone, 
        website
    )
    
    return "presentation.pptx"

In [ ]:
demo = gr.Interface(
    fn = generate_pitch_deck,
    inputs = [
        gr.Textbox(lines=2, placeholder="Write your project description here"),
        gr.Textbox(lines=2, placeholder="Write your email here"),
        gr.Textbox(lines=2, placeholder="Write your phone here"),
        gr.Textbox(lines=2, placeholder="Write your website here"),
        gr.Dropdown(choices=BASE_COLORS.keys(), label="Choose base color"),
        gr.Dropdown(choices=RESOLUTIONS, label="Choose resolution")
    ],
    outputs = "file",
)
demo.launch()